In [4]:
import pymongo
import pandas as pd
import matplotlib as plt
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures

market = futures.indexMarket
unit = futures.unit
position = futures.position

date = "20190510"

#加载数据
market = DataFrame(list(market.find({'date': {'$gte': date}})))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find({'date': {'$gte': date}})))
print(position.head())
#大写字母
position=position.rename(columns={'var':'variety'})

#删除/选取某行含有特殊数值的列
position=position.dropna()
#选择需要显示的字段

data1=market[['date','variety','set_close']]

data2=unit[[ 'variety','unit']]
position=position[['date','variety','symbol','long_party_name','long_openIntr', 'long_openIntr_chg','short_party_name','short_openIntr', 'short_openIntr_chg']]

#查询会员
members='永安期货'
data3=position[(position['long_party_name'] == members)]

#汇总合约
data3=data3[['date','variety','long_party_name','long_openIntr']]

data3=data3.groupby(['date','variety','long_party_name'])[['long_openIntr']].sum()

data4=position[(position['short_party_name'] == members)]
data4=data4[['date','variety','short_party_name','short_openIntr']]
data4=data4.groupby(['date','variety','short_party_name'])[['short_openIntr']].sum()


#并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: members,axis=1)

#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['long_openIntr']-x['short_openIntr'],axis=1)

# df['当日涨幅']=df.apply(lambda x: x['收盘']-x['收盘'].shift(1)-1,axis=1)
#选择需要显示的字段
data5=data5[['会员简称','long_openIntr','short_openIntr','净持仓']]
data5=data5.reset_index(['variety','date'])
netpostion=data5.set_index('date')

netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)
# print(netpostion)
#合约价值
contractValue=pd.merge(data1,data2,how='left',sort=False).drop_duplicates()
contractValue['contractValue'] = contractValue.apply(lambda x: x['set_close']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]

#值替换replace()
# contractValue=contractValue.replace(['TA'],'PTA')
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')

#净持仓价值
sz['净持仓价值']=sz.apply(lambda x: x['净持仓']*x['contractValue'],axis=1)
sz=sz[['date','variety','会员简称','净持仓价值']]
sz=sz.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()
sz
sz['化工板块']=sz[['RU','MA','V','L','PP','BU','TA']].sum(axis=1)
sz['油脂板块']=sz[['Y','P','OI']].sum(axis=1)
sz['有色板块']=sz[['CU','AL','ZN','NI','PB']].sum(axis=1)
sz['黑色板块']=sz[['RB','HC','ZC','J','JM','I']].sum(axis=1)
sz['黄金白银']=sz[['AU','AG']].sum(axis=1)
sz['工业品']=sz[['CU','AL','ZN','NI','PB','RB','FG','RU','L','V','TA','MA','PP','JM','J','ZC','I','BU','HC','SM','SF','FU']].sum(axis=1)
sz['农产品']=sz[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].sum(axis=1)
sz['商品板块']=sz.sum(axis=1)


chemical=sz.copy()
chemical=chemical[['RU','MA','V','L','PP','BU','TA','商品板块','工业品','化工板块']]
chemical['max']=chemical[['RU','MA','V','L','PP','BU','TA']].idxmax(axis=1)
chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
chemical



                        _id      date  long_openIntr  long_openIntr_chg  \
0  5cf3bc7c9fbea9229c5238f9  20190510         7323.0               38.0   
1  5cf3bc7c9fbea9229c5238fa  20190510         7275.0             -120.0   
2  5cf3bc7c9fbea9229c5238fb  20190510         4885.0              152.0   
3  5cf3bc7c9fbea9229c5238fc  20190510         3631.0               52.0   
4  5cf3bc7c9fbea9229c5238fd  20190510         3227.0               39.0   

  long_party_name  rank  short_openIntr  short_openIntr_chg short_party_name  \
0            中信期货     1          7867.0              -103.0             中信期货   
1            华泰期货     2          6855.0               -96.0             华泰期货   
2            光大期货     3          5003.0               217.0             光大期货   
3            银河期货     4          3652.0              -204.0             银河期货   
4            永安期货     5          3330.0               -84.0             国泰君安   

  symbol variety      vol  vol_chg vol_party_name  
0     AP      AP

,variety,RU,MA,V,L,PP,BU,TA,商品板块,工业品,化工板块,max,min
date,会员简称,,,,,,,,,,,,
20190510,永安期货,-186085900,-1414249200,-527998080,-90509370,533492400,-362316240,2706278790,13267140336,4898738120,658612400,TA,MA
20190513,永安期货,-311240710,-1486607420,-512435890,-179961250,525641875,-369247200,2167788285,5166389851,2609351958,-166062310,TA,MA
20190514,永安期货,-40361040,-1726866730,-451180680,-278339550,517394485,-272170500,1891588725,-3274022810,-884531693,-359935290,TA,MA
20190515,永安期货,-91419000,-1651912010,-400343040,-331462400,791254620,-124138150,2135968200,1165491534,872799594,327948220,TA,MA
20190516,永安期货,-201357820,-1765069950,-399043125,-432622500,972595910,-48706360,2249938900,10482803129,4233754939,375735055,TA,MA
20190517,永安期货,-165167660,-1391379500,-307747440,-418460000,1073357940,-48203880,1931710800,9240086523,4662698756,674110260,TA,MA
20190520,永安期货,-189310560,-1440133830,-332881280,-492123655,1201346100,62874000,2112362055,16896947990,7270000130,922132830,TA,MA
20190521,永安期货,-223014400,-1415188800,-369670175,-331446930,1214363040,12301500,2015191950,10973891482,6155068116,902536185,TA,MA
20190522,永安期货,-200185800,-1436910090,-347091525,-324374000,1201899035,18592200,2317725540,12174628961,6115468215,1229655360,TA,MA
